In [1]:
import sys
sys.path.append("../")
import glob
from hphc.VizCorrect import VizCorrect
%gui qt

In [2]:
Masterdir = '/Users/aimachine/Dalmiro_Laura/Correctiontooltest/'
Resultsdir = '/Users/aimachine/Dalmiro_Laura/Correctiontooltest/Results/'
BinaryName = 'BinaryWater'

MaskName = 'Mask'
VeinImageName = 'Vein'
HairName = 'Hair'
MarkerName = 'Markers'
CsvName = 'Area_Stats'
fileextension = '*tif'
doCompartment = False
max_size = 700
calibrationX = 1
calibrationY = 1
Viz = VizCorrect(Masterdir, Resultsdir,fileextension,HairName, MaskName, BinaryName, VeinImageName, MarkerName,calibrationX, calibrationY, doCompartment, max_size)


First we redo the watershed segmentation by adding/removing hair cells

In [3]:
Viz.showNapari()

/Users/aimachine/opt/anaconda3/envs/testenv/lib/python3.9/site-packages/napari/_vispy/vispy_camera.py:109: RuntimeWarning: divide by zero encountered in true_divide
  zoom = np.min(canvas_size / scale)


Correct for the segmentation Masks and obtian size statistics

In [ ]:
Viz.showsecondNapari()